# Author ID Accuracy
+ Create and train a Naive Bayes classifier in naive_bayes/nb_author_id.py
+ Use it to make predictions for the test set
+ What is the accuracy?

> + When training you may see the following error: UserWarning: Duplicate scores
+ Result may depend on feature ordering
+ There are probably duplicate features, or you used a classification score for a regression task
+ warn("Duplicate scores. Result may depend on feature ordering.")

+ This is a warning that two or more words happen to have the same usage patterns in the emails--as far as the algorithm is concerned, this means that two features are the same
+ Some algorithms will actually break (mathematically won’t work) or give multiple different answers (depending on feature ordering) when there are duplicate features and sklearn is giving us a warning
+ Good information, but not something we have to worry about

### This is the code to accompany the Lesson 1 (Naive Bayes) mini-project. 

**Use a Naive Bayes Classifier to identify emails by their authors**
```
        authors and labels:
        Sara has label 0
        Chris has label 1
```

In [26]:
import pickle
#import cPickle # http://bit.ly/2ibKHa3
import _pickle as cPickle
import numpy

from sklearn import cross_validation
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_selection import SelectPercentile, f_classif

def preprocess(words_file = "../tools/word_data.pkl", authors_file="../tools/email_authors.pkl"):
    """ 
        this function takes a pre-made list of email texts (by default word_data.pkl)
        and the corresponding authors (by default email_authors.pkl) and performs
        a number of preprocessing steps:
            -- splits into training/testing sets (10% testing)
            -- vectorizes into tfidf matrix
            -- selects/keeps most helpful features

        after this, the feaures and labels are put into numpy arrays, which play nice with sklearn functions

        4 objects are returned:
            -- training/testing features
            -- training/testing labels

    """
    ### the words (features) and authors (labels), already largely preprocessed
    ### this preprocessing will be repeated in the text learning mini-project
    authors_file_handler = open(authors_file, "rb")
    authors = pickle.load(authors_file_handler)
    authors_file_handler.close()

    words_file_handler = open(words_file, "rb")
    word_data = cPickle.load(words_file_handler)
    words_file_handler.close()
    ### test_size is the percentage of events assigned to the test set
    ### (remainder go into training)
    features_train, features_test, labels_train, labels_test = cross_validation.train_test_split(word_data, authors, test_size=0.1, random_state=42)
    ### text vectorization--go from strings to lists of numbers
    vectorizer = TfidfVectorizer(sublinear_tf=True, max_df=0.5,
                                 stop_words='english')
    features_train_transformed = vectorizer.fit_transform(features_train)
    features_test_transformed  = vectorizer.transform(features_test)
    ### feature selection, because text is super high dimensional and 
    ### can be really computationally chewy as a result
    selector = SelectPercentile(f_classif, percentile=10)
    selector.fit(features_train_transformed, labels_train)
    features_train_transformed = selector.transform(features_train_transformed).toarray()
    features_test_transformed  = selector.transform(features_test_transformed).toarray()
    ### info on the data
    print("no. of Chris training emails:", sum(labels_train))
    print("no. of Sara training emails :" , len(labels_train)-sum(labels_train))
    
    return features_train_transformed, features_test_transformed, labels_train, labels_test

In [27]:
features_train, features_test, labels_train, labels_test = preprocess()

no. of Chris training emails: 7936
no. of Sara training emails : 7884


In [28]:
#+--------------------------------------------------------------------+
#| 'features_train' are the features for the training                 |
#| 'features_test' are the testing datasets                           |
#| "labels_train" and "labels_test" are the corresponding item labels |
#+--------------------------------------------------------------------+
def NBAccuracy(features_train, labels_train, features_test, labels_test):
    from time import time
    """ Compute the accuracy of your Naive Bayes classifier """
    ### import the sklearn module for GaussianNB
    from sklearn.naive_bayes import GaussianNB
    from sklearn.metrics     import accuracy_score
    ### create classifier
    clf = GaussianNB()
    ### fit the classifier on the training features and labels
    t0 = time()
    clf.fit(features_train, labels_train)
    print("Training time  :", round(time()-t0, 3), "s")
    ### use the trained classifier to predict labels for the test features
    t0 = time()
    pred = clf.predict(features_test)
    print("Predicting time:", round(time()-t0, 3), "s")
    ### calculate and return the accuracy on the test data
    accuracy = accuracy_score(pred,labels_test)
    return accuracy



In [29]:
NBAccuracy(features_train, labels_train, features_test, labels_test)

Training time  : 1.562 s
Predicting time: 0.278 s


0.97326507394766781